* Data Validation: 
It acta as a security check point.If the NYC  API suddenly changes the column name tip_amount or if a column name tip_amount to Tip_amt, or if a column's data type changes from float to string,your pipeline should immediately stip alert you ,rather than crashing  silently hours late during themodel training phase

In [9]:
import os

In [10]:
%pwd

'e:\\dsProject\\nycTaxiProject\\research'

In [11]:
os.chdir("../")

In [18]:
%pwd

'e:\\dsProject\\nycTaxiProject'

In [ ]:
#Entity : It's a real world object with some characteristics and behavior. It can be a person, place, thing, or concept. In data science, an entity is often represented as a row in a dataset, where each column represents a characteristic or attribute of that entity.

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig: # Data Validation Configuration is a set of parameters and settings that define how data validation should be performed in a data processing pipeline. It typically includes information about the data sources, validation rules, thresholds, and any other relevant settings that guide the validation process.
    root_dir: Path
    STATUS_FILE: str
    data_dir: Path
    all_schema: dir

In [14]:
from nycTaxiProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH,SCHEMA_FILE_PATH
from nycTaxiProject.util.common import  read_yaml,create_directories,get_size

In [26]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH

            ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        create_directories([config.root_dir])

        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            data_dir=config.data_dir,
            all_schema=self.schema
        )
        return data_validation_config
    

In [16]:
import os
import pandas as pd
from logging import Logger
# from nycTaxiProject.entity

In [28]:
class Data_Validation:
    def __init__(self,config:DataValidationConfig):
        self.config=config
    
    def validation_all_columns(self)->bool:
        try:
            validation_status=None
            df=pd.read_csv("research/data.csv")
            col=list(df.columns)

            all_schema=self.config.all_schema.keys()

            for col in all_schema:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status : {validation_status}")
                    break
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status : {validation_status}")

            return validation_status
        except Exception as e:
            raise e

In [ ]:
from nycTaxiProject.config.configuration import ConfigurationManager
from nycTaxiProject.components.data_validation import Data_Validation
from nycTaxiProject import logger

In [30]:
try:
    logger.info(">>> Stage: Data Validation Started <<<")
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = Data_Validation(config=data_validation_config)
    data_validation.validation_all_columns()
    logger.info(">>> Stage: Data Validation Completed! <<<")
except Exception as e:
    logger.exception(e)
    raise e

[2026-02-21 21:47:00,440 : INFO : 3788650901 : >>> Stage: Data Validation Started <<<]
[2026-02-21 21:47:00,449 : INFO : common : yaml file: config\config.yaml LOADED successfully]
[2026-02-21 21:47:00,456 : INFO : common : yaml file: params.yaml LOADED successfully]
[2026-02-21 21:47:00,476 : INFO : common : yaml file: schema.yaml LOADED successfully]
[2026-02-21 21:47:00,481 : INFO : common : created directory at: artifacts]
[2026-02-21 21:47:00,489 : INFO : common : created directory at: artifacts/data_validation]
[2026-02-21 21:47:06,140 : INFO : 3788650901 : >>> Stage: Data Validation Completed! <<<]


In [4]:
import pandas as pd
df=pd.read_csv("NYc low space data.csv")

In [5]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 19 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   vendorid               500000 non-null  int64  
 1   tpep_pickup_datetime   500000 non-null  str    
 2   tpep_dropoff_datetime  500000 non-null  str    
 3   passenger_count        500000 non-null  float64
 4   trip_distance          500000 non-null  float64
 5   ratecodeid             500000 non-null  float64
 6   store_and_fwd_flag     500000 non-null  str    
 7   pulocationid           500000 non-null  int64  
 8   dolocationid           500000 non-null  int64  
 9   payment_type           500000 non-null  int64  
 10  fare_amount            500000 non-null  float64
 11  extra                  500000 non-null  float64
 12  mta_tax                500000 non-null  float64
 13  tip_amount             500000 non-null  float64
 14  tolls_amount           500000 non-null  float64

In [7]:
df.head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01T00:32:10.000,2023-01-01T00:40:36.000,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01T00:55:08.000,2023-01-01T01:01:27.000,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01T00:25:04.000,2023-01-01T00:37:49.000,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01T00:03:48.000,2023-01-01T00:13:25.000,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01T00:10:29.000,2023-01-01T00:21:19.000,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
